In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import pandas as pd
import numpy as np
import warnings

# Data read
train = pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv')
test = pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv')
sub = pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/sample_submission.csv')

display(train.head())
display(test.head())
display(sub.head())

# Data 수, 속성 
print('train shape: ', train.shape)
print('test shape: ', test.shape)
print('sub shape: ', sub.shape)

# Null 값 확인
display(train.isna().sum())
display(test.isna().sum())

# 중복 row 확인
display(train.duplicated().sum())
display(test.duplicated().sum())


 # 이를 HTML 을 통해 테이블로 생성/출력
from IPython.core.display import HTML

# 칼럼 별 value_counts 값을 List로 출력하기 위한 함수 생성.
def value_counts_all(df, columns):
    pd.set_option('display.max_rows', 100) 
    table_list = []
     # 칼럼 별 value_counts 값을 List로 만드는 for문
    for col in columns:
        table_list.append(pd.DataFrame(df[col].value_counts()))
    return HTML( 
        f"<table><tr> {''.join(['<td>' + table._repr_html_() + '</td>' for table in table_list])} </tr></table>")  

# 함수호출
value_counts_all(train, ['anchor', 'target', 'context', 'score'])
value_counts_all(test, ['anchor', 'target', 'context'])

# anchor 칼럼의 target 값 count. anchor의 중복이 많음을 확인.
g_train_df = train.groupby(["anchor"])["target"].count().reset_index().sort_values(["target"], ascending=False)
print(g_train_df)


# 대분류 정의
context_dict = {
    'A': 'Human Necessities',
    'B': 'Operations and Transport',
    'C': 'Chemistry and Metallurgy',
    'D': 'Textiles',
    'E': 'Fixed Constructions',
    'F': 'Mechanical Engineering',
    'G': 'Physics',
    'H': 'Electricity',
    'Y': 'Emerging Cross-Sectional Technologies'
}

# context 칼럼 데이터의 길이 Max 확인, 대분류로 Context를 나눠 보려는 의도.
train['context'].str.len().max()



In [ ]:
# title Data read 참고 : https://www.uspto.gov/web/patents/classification/cpc/html/cpc-A.html
cpc_codes_df = pd.read_csv("../input/titles/titles.csv")
cpc_codes_df.head(100)


In [ ]:

# Data 수, 속성 
cpc_codes_df.shape

# anchor 칼럼의 target 값 count. anchor의 중복이 많음을 확인.
g_train_df = train.groupby(["anchor"])["target"].count().reset_index().sort_values(["target"], ascending=False)
print(g_train_df)

!pip install fuzzywuzzy

# 함수 생성, 유사도 값을 더하여 칼럼을 생성함.
def create_feature(df, cpc_codes_df):
    import fuzzywuzzy
    from fuzzywuzzy import fuzz
    from fuzzywuzzy import process
    
    df['section'] = df['context'].str[:1] # context의 대분류를 section 으로 분류.
    df['class'] = df['context'].str[1:] # context의 숫자를 class로 분류
    
    df['anchor_len'] = df['anchor'].apply(lambda x: len(x.split(' '))) # 띄어쓰기 기준으로 단어의 갯수 분류
    df['target_len'] = df['target'].apply(lambda x: len(x.split(' '))) # 띄어쓰기 기준으로 단어의 갯수 분류
    
    pattern = '[0-9]' 
    mask = df['anchor'].str.contains(pattern, na=False) # 숫자 들어갔는지 확인 T/F
    df['num_anchor'] = mask
    mask = df['target'].str.contains(pattern, na=False) # 숫자 들어갔는지 확인 T/F
    df['num_target'] = mask

     #                   context 칼럼과 cpc_codes_df의 code값이 같을 때  Title 칼럼값을 출력. 소문자로.
    df['context_desc'] = df['context'].map(cpc_codes_df.set_index('code')['title']).str.lower()
    
    fuzzy_anchor_target_scores = []
    fuzzy_anchor_context_scores = []
    fuzzy_taget_context_scores = []

    # FUZZYWUZZY 라이브러리 : 두 문자열 간의 Levenshtein Distance 계산. 편집거리, 두 문자열이 얼마나 유사한지 나타내는 알고리즘. 
    for index, row in df.iterrows():
        # fuzz.ratio : 부분비율 사용(최적 부분 논리 메커니즘 사용), 하나의 짧은 문자열이 다른 긴 문자열의 일부인 경우 부분 비율 함수를 사용
        fuzzy_anchor_target_scores.append(fuzz.ratio(row['anchor'], row['target'])) # 입력 df 의 anchor과 target 의 유사도.
        fuzzy_anchor_context_scores.append(fuzz.ratio(row['anchor'], row['context_desc'])) # 입력 df 의 anchor과 df의 context의 값의 유사도
        fuzzy_taget_context_scores.append(fuzz.ratio(row['context_desc'], row['target'])) #  df의 context의 값과 입력 df 의 target의 유사도
    df['fuzzy_at_score'] = fuzzy_anchor_target_scores
    df['fuzzy_ac_score'] = fuzzy_anchor_context_scores
    df['fuzzy_tc_score'] = fuzzy_taget_context_scores
    df['fuzzy_c_score'] = df['fuzzy_ac_score'] + df['fuzzy_tc_score'] # df의 anchor -  df의 context의 값 유사도 + df의 target - context 유사도 
    df['fuzzy_total'] = df['fuzzy_at_score'] + df['fuzzy_c_score'] # df의 anchor - df의 target 유사도 + (df의 anchor - df의 context의 값 유사도 + df의 target - context 유사도)
    
    df.drop(['context', 'fuzzy_ac_score', 'fuzzy_tc_score'], 1, inplace=True)
    
    return df

new_train = create_feature(train.copy(), cpc_codes_df)
new_test = create_feature(test.copy(), cpc_codes_df)
new_train.head()

!pip install fuzzywuzzy

# 함수 생성, 유사도 값을 더하여 칼럼을 생성함.
def create_feature2(df, cpc_codes_df):
    import fuzzywuzzy
    from fuzzywuzzy import fuzz
    from fuzzywuzzy import process
    
    df['section'] = df['context'].str[:1] # context의 대분류를 section 으로 분류.
    df['class'] = df['context'].str[1:] # context의 숫자를 class로 분류
    
    df['anchor_len'] = df['anchor'].apply(lambda x: len(x.split(' '))) # 띄어쓰기 기준으로 단어의 갯수 분류
    df['target_len'] = df['target'].apply(lambda x: len(x.split(' '))) # 띄어쓰기 기준으로 단어의 갯수 분류
    
    pattern = '[0-9]' 
    mask = df['anchor'].str.contains(pattern, na=False) # 숫자 들어갔는지 확인 T/F
    df['num_anchor'] = mask
    mask = df['target'].str.contains(pattern, na=False) # 숫자 들어갔는지 확인 T/F
    df['num_target'] = mask

     #                   context 칼럼과 cpc_codes_df의 code값이 같을 때  Title 칼럼값을 출력. 소문자로.
    df['context_desc'] = df['context'].map(cpc_codes_df.set_index('code')['title']).str.lower()
    
    fuzzy_anchor_target_scores = []
    fuzzy_anchor_context_scores = []
    fuzzy_taget_context_scores = []

    # FUZZYWUZZY 라이브러리 : 두 문자열 간의 Levenshtein Distance 계산. 편집거리, 두 문자열이 얼마나 유사한지 나타내는 알고리즘. 
    for index, row in df.iterrows():
        # fuzz.ratio : 토큰 정렬 비율 사용.
        fuzzy_anchor_target_scores.append(fuzz.token_sort_ratio(row['anchor'], row['target'])) # 입력 df 의 anchor과 target 의 유사도.
        fuzzy_anchor_context_scores.append(fuzz.token_sort_ratio(row['anchor'], row['context_desc'])) # 입력 df 의 anchor과 df의 context의 값의 유사도
        fuzzy_taget_context_scores.append(fuzz.token_sort_ratio(row['context_desc'], row['target'])) #  df의 context의 값과 입력 df 의 target의 유사도
    df['fuzzy_at_score'] = fuzzy_anchor_target_scores
    df['fuzzy_ac_score'] = fuzzy_anchor_context_scores
    df['fuzzy_tc_score'] = fuzzy_taget_context_scores
    df['fuzzy_c_score'] = df['fuzzy_ac_score'] + df['fuzzy_tc_score'] # df의 anchor -  df의 context의 값 유사도 + df의 target - context 유사도 
    df['fuzzy_total'] = df['fuzzy_at_score'] + df['fuzzy_c_score'] # df의 anchor - df의 target 유사도 + (df의 anchor - df의 context의 값 유사도 + df의 target - context 유사도)
    
    df.drop(['context', 'fuzzy_ac_score', 'fuzzy_tc_score'], 1, inplace=True)
    
    return df

new_train2 = create_feature2(train.copy(), cpc_codes_df)
new_test2 = create_feature2(test.copy(), cpc_codes_df)
new_train2.head()

!pip install --upgrade matplotlib

import matplotlib.pyplot as plt
import seaborn as sns

# 그래프 출력하기 위한 sns 
sns.set()
fig, ax = plt.subplots(figsize=(16, 8))
sns.countplot(data=new_train2, x='section', ax=ax) # 새로운 유사도를 게산한 Train df의 section 별 count
ax.set_xticklabels([context_dict['A'], context_dict['C'], context_dict['F'], context_dict['H'], context_dict['B'], 
                    context_dict['D'], context_dict['E'], context_dict['G']], rotation=45);

fig, ax = plt.subplots(figsize=(16, 8))
sns.countplot(data=new_train, x='class', ax=ax); # # 새로운 유사도를 게산한 Train df의 class별 conut

fig, ax = plt.subplots(figsize=(16, 8))
# anchor 의 단어갯수에 대한 커널 밀도 추정(최대값, 최소값은 어떤지, 어떤 모양으로 얼마나 퍼져 있는지 분포가 확인)
sns.kdeplot(data=new_train, x='anchor_len', ax=ax);

fig, ax = plt.subplots(figsize=(16, 8))
# target 의 단어갯수에 대한 커널 밀도 추정(최대값, 최소값은 어떤지, 어떤 모양으로 얼마나 퍼져 있는지 분포가 확인)
sns.kdeplot(data=new_train, x='target_len', ax=ax);

fig, ax = plt.subplots(figsize=(16, 8))

# anchor의 숫자 유무에 대한 conut 
sns.countplot(data=new_train, x='num_anchor', ax=ax);
for container in ax.containers:
    ax.bar_label(container) # 버전 upgrade 필요 런타임 다시시작 > 모두실행 
    
fig, ax = plt.subplots(figsize=(16, 8))
# target의 숫자 유무에 대한 conut 
sns.countplot(data=new_train, x='num_target', ax=ax);
for container in ax.containers:
    ax.bar_label(container)
    
#Score Relationship

temp_train = new_train2.copy() # create_feature 사용한 테이블 복사
# 테이블 칼럼 2개 정의 
temp_train['score_jitter'] = new_train['score'] + np.random.normal(0, 0.1, size=len(new_train['score'])) # 정규 (가우시안) 분포에서 랜덤 표본을 추출, 평균0, 표준편차 0.1, 사이즈 테이블크기
temp_train['fuzzy_at_jitter'] = new_train['fuzzy_at_score'] + np.random.normal(0, 0.5, size=len(new_train['score'])) # 트레이닝 테이블의 anchor과 target 의 유사도(anchor가 target에 얼마나 포함되는지) + 랜덤표본

print(temp_train['score_jitter'])
print(temp_train['fuzzy_at_jitter'])

#Score Relationship

temp_train2 = new_train2.copy() # create_feature 사용한 테이블 복사
# 테이블 칼럼 2개 정의 
temp_train2['score_jitter'] = new_train2['score'] + np.random.normal(0, 0.1, size=len(new_train2['score'])) # 정규 (가우시안) 분포에서 랜덤 표본을 추출, 평균0, 표준편차 0.1, 사이즈 테이블크기
temp_train2['fuzzy_at_jitter'] = new_train2['fuzzy_at_score'] + np.random.normal(0, 0.5, size=len(new_train2['score'])) # 트레이닝 테이블의 anchor과 target 의 유사도(anchor가 target에 얼마나 포함되는지) + 랜덤표본

print(temp_train2['score_jitter'])
print(temp_train2['fuzzy_at_jitter'])


def regplot_with_corr(df, x, y, ax=None):
    from matplotlib.offsetbox import AnchoredText
    if ax==None:
        fig, ax = plt.subplots(figsize=(16, 8)) #figure 및 axes 객체를 포함하는 튜플을 반환하는 함수
        
    # scatter 플롯(산점도) x,y축의 값을 산점도로 표현
    scatter_kws = dict( # 키=값 형식으로 딕셔너리를 만듦
                alpha=0.1, # 투명도
                s=4, # size
            )
    line_kws = dict(color='red') 
    corr = df[x].corr(df[y]) #열 간의 모든 쌍별 상관 관계가 반환
    sns.regplot(data=df, x=x, y=y, 
                scatter_kws=scatter_kws, 
                line_kws=line_kws,  # 라인색
                ax=ax)
    at = AnchoredText( #Textbox 
                f"{corr:.2f}", # 표시 될 값, 소수점 둘째자리 반올림(.2f)
                prop=dict(size="large"), #글꼴 크기의 일부로 텍스트 주변을 채우는 것
                frameon=True, # 라인 표시
                loc="upper left", # 위치를 자동으로 지정
            )
    at.patch.set_boxstyle("square, pad=0.0") # box 스타일
    ax.add_artist(at) # Textbox add

 # train 테이블의 산점도 표시
regplot_with_corr(temp_train2, 'score_jitter', 'fuzzy_at_jitter') # X: Score + 랜덤 표본 , Y : 트레이닝 테이블의 anchor과 target 의 유사도 + 랜덤표본
# 결과 : 선형 그래프 - score가 높을수록 트레이닝 테이블의 anchor과 target 의 유사도가 높다.

 # train 테이블의 산점도 표시
regplot_with_corr(temp_train2, 'score_jitter', 'fuzzy_at_jitter') # X: Score + 랜덤 표본 , Y : 트레이닝 테이블의 anchor과 target 의 유사도 + 랜덤표본
# 결과 : 선형 그래프 - score가 높을수록 트레이닝 테이블의 anchor과 target 의 유사도가 높다.

regplot_with_corr(temp_train2, 'score_jitter', 'fuzzy_c_score')  
# X: Score + 랜덤 표본
# Y : 트레이닝 테이블의 anchor, 트레이닝 테이블의 context의 값 유사도 + 트레이닝 테이블의 target, context 값의 유사도 = anchor~context(title text)~target 간의 유사도

# 결과 : 선형 그래프 - score와 anchor~context(title text)~target 간의 유사도는 별 관계가 없다

regplot_with_corr(temp_train2, 'score_jitter', 'fuzzy_total') 
# X : Score + 랜덤 표본
# Y : 트레이닝 테이블의 anchor과 target 의 유사도 + (anchor~context(title text)~target 간의 유사도) 

# 결과 : 선형 그래프 - score와 anchor~context(title text)~target 간의 유사도는 별 관계가 없지만, 
#        트레이닝 테이블의 anchor과 target 의 유사도를 더하면 첫 그래프와 비슷해진다.

fig, ax = plt.subplots(figsize=(16, 8)) # 그래프 사이즈
temp_train2['score_jitter'] = new_train2['score'] + np.random.normal(0, 0.1, size=len(new_train2['score'])) # score + 랜덤표본
temp_train2['fuzzy_at_jitter'] = new_train2['fuzzy_at_score'] + np.random.normal(0, 0.5, size=len(new_train2['score']))  # 트레이닝 테이블의 anchor과 target 의 유사도 + 랜덤표본

sns.scatterplot(data=temp_train2, x='score_jitter', y='fuzzy_at_jitter', ax=ax, 
                alpha=0.5, # 투명도
                s=10, # 첫 그래프 보다 size 4 > 10 으로 키움
                hue='fuzzy_total'); #fuzzy_total 값에 따른 색 변화

# 결과 : 선형 그래프 - score가 높을수록 트레이닝 테이블의 anchor과 target 의 유사도가 높고, 높을수록 fuzzy_total 값도 크다.


In [ ]:

numeric_cols = new_train2.select_dtypes(include=np.number).columns.tolist() # np.number 데이터형인 칼럼 List 로 반환
print(numeric_cols, '\n')

print(set(new_train2.columns), '\n')
print((set(new_train2.columns) - set(numeric_cols)), '\n')
object_cols = list(set(new_train2.columns) - set(numeric_cols)) #  데이터형이 np.number인 칼럼을 제외하여 List로 반환

numeric_cols.remove("score") # score 삭제
ignore_cols = ['id']

print('numerical features: ', numeric_cols) # 수치형 변수 
print('object features: ', object_cols) # 범주형  변수 

!pip install catboost
from catboost import CatBoostRegressor
# Catboost는 Level-wise 로 트리를 만든다.
# Catboost 는 일부만 가지고 잔차계산을 한 뒤, 이걸로 모델을 만들고, 그 뒤에 데이터의 잔차는 이 모델로 예측한 값을 사용한다.
# 데이터 대부분이 수치형 변수인 경우, Light GBM 보다 학습 속도가 느리다. (즉 대부분이 범주형 변수인 경우 쓰라는 말)
# 이번 분석의 경우 context (ex. A01) 을 section과 class로 나눔. 이게 범주형 변수라는 듯.

cat_base = CatBoostRegressor(
    ignored_features=ignore_cols, # 분석에서 무시해야하는 기능
    cat_features=object_cols, # 범주 형 열이있는 변수 배열
    # eval_metric = 과적 합을 감지하는 데 사용되는 메트릭 **성능 측정 지표**
    eval_metric='MAE' # MAE : mean absolute error = 실제 값과 예측 값의 차이(Error)를 절대값으로 변환해 평균화. 
)


X_train2 = new_train2.drop(['score'], 1) # train 에서 score 칼럼 삭제
y_train2 = new_train2['score'] # train 에서의 score 칼럼
cat_base.fit(X_train2, y_train2, silent=True) # train 테이블로 score와 다른칼럼들로 catboost를 fit으로 훈련시킴.

X_test2 = new_test2.copy() # test 데이터셋 생성
# fit으로 훈련을 하면 predict 함수를 사용해 예측을 만든다.
preds2 = pd.DataFrame(cat_base.predict(X_test2), columns=['preds']) # x에 test를 넣어 나온 데이터 score값이 preds로 변환
preds2.head()

sub['score'] = preds2['preds'] # sub 데이터셋의 score칼럼 생성.
sub.to_csv('submission.csv', index=False) # 결과 파일로 저장
sub.head()

print(cat_base.get_feature_importance())
print()

X_train = new_train.drop(['score'], 1) # train 에서 score 칼럼 삭제
y_train = new_train['score'] # train 에서의 score 칼럼
cat_base.fit(X_train, y_train, silent=True) # train 테이블로 score와 다른칼럼들로 catboost를 fit으로 훈련시킴.

X_test = new_test.copy() # test 데이터셋 생성
# fit으로 훈련을 하면 predict 함수를 사용해 예측을 만든다.
preds = pd.DataFrame(cat_base.predict(X_test), columns=['preds2']) # x에 test를 넣어 나온 데이터 score값이 preds로 변환
preds.head()

sub['score2'] = preds['preds2'] # sub 데이터셋의 score칼럼 생성.
sub.to_csv('submission2.csv', index=False) # 결과 파일로 저장
sub.head()

print(cat_base.get_feature_importance())
print()

# 그래프 그리는 함수 생성
def plot_feature_importance(importance # 기능 중요도를 계산한 배열이 들어올 것
                            ,names # 연관된 칼럼명이 들어올 것
                            ,model_type): # CATBOOST
    
    #기능 중요도 및 칼럼명으로 배열 생성
    feature_importance = np.array(importance) # 기능 중요도를 계산한 배열
    feature_names = np.array(names) # 연관된 칼럼명
    
    #Dictionary를 사용해 DataFrame 생성
    data={'feature_names':feature_names,'feature_importance':feature_importance} 
    fi_df = pd.DataFrame(data)
    
    #Sort 
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    
    plt.figure(figsize=(10,8)) # 그래프 크기 지정
    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names']) # x,y축 지정

    #Add chart labels
    plt.title(model_type + 'FEATURE IMPORTANCE')
    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')
    
plot_feature_importance(cat_base.get_feature_importance() # 기능 중요도 를 계산하고 반환 : fit돌린 훈련의 결과
                       ,X_train.columns # fit 돌린 train 테이블의 칼럼들 을 Y 축으로 
                       ,'CATBOOST') 

!pip install shap
import shap

explainer = shap.TreeExplainer(cat_base) # Tree model Shap Value 확인 객체(cat_base : 훈련모델)지정
shap_values = explainer.shap_values(X_test) # test 데이터셋으로 훈련모델 Shap Values 계산
shap.summary_plot(shap_values, X_test) #모든 변수들의 shap value를 요약한 것
# 해당 변수가 빨간색을 띄면 target(price)에 대해 양의 영향력이 존재하는 것이고, 
# 파란색을 띄면 음의 영향력이 존재하는 것

shap.initjs() # javascript 초기화 (graph 초기화)
shap.force_plot(explainer.expected_value # 기대값
               ,shap_values # 전체 검증 데이터 셋에 대해서 적용
               ,X_test)  #test 데이터셋

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session